In [2]:
## Import Packages 
## Download required modules

## Data manipulation
import pandas as pd
import numpy as np
import random as rd

## Deep Learning architecture
import torch
from torch import nn
import torchvision
#print(torch.__version__)

## Modelling Analysis
import sklearn as sk
import torchvision.models
import torchvision.transforms as transforms
from torchvision.transforms import Lambda
from PIL import Image

## Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
## Models to download baesed on image input size 

## performance may be influenced as the input layer xforms the image to 256 then crops them to 224 
resnet18


## Minimum image size is 32 x 32 for usage 
vgg16 

/Users/james/.pyenv/versions/3.11.9/envs/env_ilab/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [12]:
## Freeze Parameters and add FC - MobileNet
!pip install torchinfo
from torchinfo import summary
for params in mobile_net.parameters():
    params.requires_grad = False

num_classes = 2 ## output classes

mobile_net.classifier = nn.Sequential(
            nn.Linear(960, 1280),
            nn.ReLU(),
            nn.Linear(1280, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes))

sum = summary(mobile_net,
        input_size=(128, 3, 256, 256), # make sure this is "input_size", not "input_shape" (batch_size, color_channels, height, width)
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)
print(sum)


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
MobileNetV3 (MobileNetV3)                                    [128, 3, 256, 256]   [128, 2]             --                   Partial
├─Sequential (features)                                      [128, 3, 256, 256]   [128, 960, 8, 8]     --                   False
│    └─Conv2dNormActivation (0)                              [128, 3, 256, 256]   [128, 16, 128, 128]  --                   False
│    │    └─Conv2d (0)                                       [128, 3, 256, 256]   [128, 16, 128, 128]  (432)                False
│    │    └─BatchNorm2d (1)                                  [128, 16, 128, 128]  [128, 16, 128, 128]  (32)                 False
│    │    └─Hardswish (2)                                    [128, 16, 128, 128]  [128, 16, 128, 128]  --

In [ ]:
#%% Final prep on Data and build Dataloader
## Specific to googlenet reqts || OHE targets/ labels

from torchvision.transforms import Lambda

T_form = transforms.Compose([transforms.Resize((256,256)),
                                transforms.ToTensor(), #standardise to 0-1
                                transforms.Normalize(
                                    mean = [0.485, 0.456, 0.406],
                                    std = [0.229, 0.224, 0.225])]) ## Taken from pytorch documentation

trg_101 = (root = './data/clin_0',
                                          split = 'train',
                                          transform = T_form,
                                          target_transform = Lambda(lambda y: torch.zeros(101, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1)),
                                          download = True)

test_101 = torchvision.datasets.Food101(root = './data',
                                          split = 'test',
                                          transform = T_form,
                                          target_transform = Lambda(lambda y: torch.zeros(101, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1)),
                                          download = True)

transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize
